In [ ]:
!pip install unidecode

In [ ]:
import pandas as pd
import boto3
import json
from preprocess_utils import preprocess_text
import unidecode

In [ ]:
session = boto3.Session()
s3 = session.client("s3")

In [ ]:
def get_keys_from_prefix(session, bucket, prefix):
    """
    :param prefix: string, prefix to search for the keys
    :return: list, list of keys
    """
    keys = []

    try:
        objects = session.list_objects(Bucket=bucket, Prefix=prefix)
        is_truncated = objects["IsTruncated"]

        for i in objects["Contents"]:
            keys.append(i["Key"])

        if is_truncated:
            last_key = keys[-1]

            while is_truncated:
                objs = session.list_objects(Bucket=bucket,
                                            Prefix=prefix, 
                                            Marker=last_key)
                is_truncated = objs["IsTruncated"]

                for i in objs["Contents"]:
                    keys.append(i["Key"])

                if is_truncated:
                    last_key = keys[-1]
    except KeyError:
        print("Prefix doesn't exist")
        keys = None

    return keys

In [ ]:
bucket = "cc-news"

In [ ]:
countries = [".ar", ".bo", ".br", ".cl", ".cr", ".co", ".gt", ".hn", ".ht", ".jm", ".mx", ".ni", ".pa", ".pe", ".pr", 
".py", ".sv", ".uy", ".ve", ".com"]

In [ ]:
for country in countries:
    print(country)
    prefix = f"processed_2/{country}/"
    keys = get_keys_from_prefix(s3, bucket, prefix)
    
    total_result = ""
    total_k = len(keys)
    total_words = 0
    total_characters = 0

    for k in keys:
        total_k -= 1
        if total_k % 100 == 0:
            print(f"total remaining:{total_k}")
        response = s3.get_object(Bucket=bucket, Key=k)
        response = response["Body"].read().decode("utf8")
        result = preprocess_text(response)

        words_k = len(result.split(" "))
        total_words += words_k
        characters_k = len(result)
        total_characters += characters_k

        total_result += result

    output_key = f"processed_3/{country}/processed.txt"
    s3.put_object(Body=total_result,
                  Bucket=bucket, 
                  Key=output_key)
